In [2]:
using ParallelTemperingMonteCarlo
using Random


In [3]:
# using Pkg
# Pkg.add("BenchmarkTools")
using BenchmarkTools

In [4]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 20000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

In [24]:
testvec = []

for j in 1:5
    tt = zeros(5)
    push!(testvec,tt)
end
indices = Array(1:5)
function mutearray(index,vector)
    vec[index] +=1
end

mutearray(indices[1],testvec[1])
broadcast(mutearray,indices,testvec)
testvec

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [10]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.00034224645072504614, 0.0003455660633384812, 0.0003586503581991548, 0.00036990359099924427, 0.0003803982593610761, 0.00045627696191242567, 0.0004195162094081378, 0.0004771578466950622, 0.00047275735439122787, 0.0005248093840288748, 0.0004995957314219647, 0.0005889027794678553, 0.0007573597256781532, 0.0006579211970061152, 0.0007477559631345421, 0.0010802942126791077, 0.0009661836407038012, 0.0014193757612390807, 0.0010035360936692008, 0.0012674214587078355, 0.0018429344718856237, 0.002965415079065907, 0.0023640644103088524, 0.0026803461550051083, 0.003186790801170793, 0.003952459084642581, 0.0033873284402009935, 0.003319541495458708, 0.0026370960977762535, 0.0021253263821210617, 0.0024132794149659604, 0.002459565431564028]
done


In [36]:
sum(results.en_histogram[1])

20000.0

In [42]:
function find_hist_index(mc_state,results,delta_en_hist)
    hist_index = floor(Int,(mc_state.en_tot - results.en_min)/delta_en_hist ) +1

    if hist_index < 1
        return 1
    elseif hist_index > results.n_bin
        return results.n_bin+2
    else
        return hist_index +1
    end
end
"""
    update_one_histval!(histogram,index)
To be used in conjunction with the find index function above to correctly update the overall results.en_histogram vector of vectors. See the updat_histogram! function defined below.
"""
function update_one_histval!(histogram,index)
    histogram[index] += 1
end

function update_histogram!(mc_states,results,delta_en_hist)
    indices = find_hist_index.(mc_states,Ref(results),Ref(delta_en_hist))
    broadcast(update_one_histval!,results.en_histogram,indices)
end
# delta_en_hist = (results.en_max - results.en_min)/(results.n_bin - 1)
# update_histogram!(mc_states,results,delta_en_hist)

# sum(results.en_histogram[1])
results.en_histogram[1][13]
# find_hist_index(mc_states[1],results,delta_en_hist)

update_histogram! (generic function with 1 method)

To test this afternoon: write new ptmc_cycle function and benchmark against existing function

In [16]:
import ParallelTemperingMonteCarlo.MCSampling.sampling_step!

In [26]:
@benchmark ptmc_cycle!($mc_states,$results,move_strat,mc_params,pot,ensemble,55,55,0,0,false,false,1000,pwd(),delta_en_hist=delta_en_hist)

BenchmarkTools.Trial: 9108 samples with 1 evaluation.
 Range (min … max):  446.809 μs …   3.872 ms  ┊ GC (min … max): 0.00% … 83.09%
 Time  (median):     484.043 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   547.293 μs ± 269.199 μs  ┊ GC (mean ± σ):  4.69% ±  8.17%

  █                                                              
  █▆▂▃▇▃▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  447 μs           Histogram: frequency by time          2.9 ms <

 Memory estimate: 701.02 KiB, allocs estimate: 3909.

In [28]:
delta_en_hist = (results.en_max - results.en_min) / (results.n_bin - 1)

3.083946926850007e-5

In [29]:
function testptmc_cycle(mc_states,results,move_strat,mc_params,pot,ensemble,n_steps,a,v,r,delta_en_hist,delta_r2)
    mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
    mc_states,results = sampling_step!(mc_params,mc_states,1000,results,delta_en_hist,delta_r2)
    if rem(i,mc_params.n_adjust) == 0
        broadcast(mc_states,Ref(mc_params.n_adjust),Ref(a),Ref(v),Ref(r))
    end

    return mc_states,results
end

delta_r2 = 4*mc_states[1].config.bc.radius2/(results.n_bin*5)

0.8085575246693822

In [32]:
testptmc_cycle(mc_states,results,move_strat,mc_params,pot,ensemble,55,55,0,0,delta_en_hist,delta_r2)

BoundsError: BoundsError: attempt to access 500-element Vector{Float64} at index [0]

3.083946926850007e-5